Обучение модели на основе кода Лиховидова.  
Тройная классификация Up, Flat, Down.

In [11]:
import sqlite3
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np
import pandas as pd
from pathlib import Path
from torch.utils.data import Dataset, DataLoader

# === 1. ЗАГРУЗКА ДАННЫХ ===
db_path = Path(r'C:\Users\Alkor\gd\data_quote_db\RTS_futures_options_day.db')

with sqlite3.connect(db_path) as conn:
    df_fut = pd.read_sql_query(
        "SELECT TRADEDATE, OPEN, LOW, HIGH, CLOSE, VOLUME FROM Futures",
        conn
    )

# === 2. КОДИРОВАНИЕ СВЕЧЕЙ (ЛИХОВИДОВ) ===
def encode_candle(row):
    open_, low, high, close = row['OPEN'], row['LOW'], row['HIGH'], row['CLOSE']
    
    direction = 1 if close > open_ else (0 if close < open_ else 2)
    upper_shadow = high - max(open_, close)
    lower_shadow = min(open_, close) - low
    body = abs(close - open_)

    def classify_shadow(shadow, body):
        return 0 if shadow < 0.1 * body else (1 if shadow < 0.5 * body else 2)

    return f"{direction}{classify_shadow(upper_shadow, body)}{classify_shadow(lower_shadow, body)}"

df_fut['CANDLE_CODE'] = df_fut.apply(encode_candle, axis=1)

# === 3. ПОДГОТОВКА ДАННЫХ ДЛЯ LSTM ===
unique_codes = sorted(df_fut['CANDLE_CODE'].unique())
code_to_int = {code: i for i, code in enumerate(unique_codes)}
df_fut['CANDLE_INT'] = df_fut['CANDLE_CODE'].map(code_to_int)

window_size = 20  
predict_offset = 1  

X, y = [], []
for i in range(len(df_fut) - window_size - predict_offset):
    X.append(df_fut['CANDLE_INT'].iloc[i:i+window_size].values)
    
    future_price = df_fut['CLOSE'].iloc[i+window_size+predict_offset]
    current_price = df_fut['CLOSE'].iloc[i+window_size]
    change = (future_price - current_price) / current_price  

    if change > 0.005:  # Рост > 0.5%
        y.append(2)  # +1 (сильный рост)
    elif change < -0.005:  # Падение > 0.5%
        y.append(0)  # -1 (сильное падение)
    else:
        y.append(1)  # 0 (флэт)

X, y = np.array(X), np.array(y)

split = int(0.8 * len(X))
X_train, y_train = X[:split], y[:split]
X_test, y_test = X[split:], y[split:]

class CandlestickDataset(Dataset):
    def __init__(self, X, y):
        self.X = torch.tensor(X, dtype=torch.long)
        self.y = torch.tensor(y, dtype=torch.long)  # Теперь метки - целые числа (0, 1, 2)

    def __len__(self):
        return len(self.X)

    def __getitem__(self, idx):
        return self.X[idx], self.y[idx]

train_dataset = CandlestickDataset(X_train, y_train)
test_dataset = CandlestickDataset(X_test, y_test)

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

# === 4. LSTM-МОДЕЛЬ С 3-КЛАССОВОЙ КЛАССИФИКАЦИЕЙ ===
class CandleLSTM(nn.Module):
    def __init__(self, vocab_size, embedding_dim, hidden_dim, output_dim):
        super(CandleLSTM, self).__init__()
        self.embedding = nn.Embedding(vocab_size, embedding_dim)
        self.lstm = nn.LSTM(embedding_dim, hidden_dim, batch_first=True)
        self.fc = nn.Linear(hidden_dim, output_dim)
        self.softmax = nn.Softmax(dim=1)

    def forward(self, x):
        x = self.embedding(x)
        x, _ = self.lstm(x)
        x = self.fc(x[:, -1, :])  
        return self.softmax(x)  # Выдает вероятности 3 классов

# === 5. ОБУЧЕНИЕ С СОХРАНЕНИЕМ ЛУЧШЕЙ МОДЕЛИ ===
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

model = CandleLSTM(vocab_size=len(unique_codes), embedding_dim=8, hidden_dim=32, output_dim=3).to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

best_accuracy = 0  
model_path = "best_model_3.pth"

epochs = 2000
for epoch in range(epochs):
    model.train()
    total_loss = 0
    for X_batch, y_batch in train_loader:
        X_batch, y_batch = X_batch.to(device), y_batch.to(device)

        optimizer.zero_grad()
        y_pred = model(X_batch)
        loss = criterion(y_pred, y_batch)
        loss.backward()
        optimizer.step()

        total_loss += loss.item()

    # Проверка на тесте
    model.eval()
    correct = 0
    total = 0
    with torch.no_grad():
        for X_batch, y_batch in test_loader:
            X_batch, y_batch = X_batch.to(device), y_batch.to(device)
            y_pred = model(X_batch).argmax(dim=1)  # Класс с наибольшей вероятностью
            correct += (y_pred == y_batch).sum().item()
            total += y_batch.size(0)

    accuracy = correct / total
    print(f"Epoch {epoch+1}/{epochs}, Loss: {total_loss/len(train_loader):.4f}, Test Accuracy: {accuracy:.2%}")

    # Сохранение лучшей модели
    if accuracy > best_accuracy:
        best_accuracy = accuracy
        torch.save(model.state_dict(), model_path)
        print(f"✅ Model saved with accuracy: {best_accuracy:.2%}")

# === 6. ПРЕДСКАЗАНИЕ С 3 КЛАССАМИ ===
model.load_state_dict(torch.load(model_path))
model.eval()

last_sequence = torch.tensor(df_fut['CANDLE_INT'].iloc[-20:].values, dtype=torch.long).unsqueeze(0).to(device)

with torch.no_grad():
    probabilities = model(last_sequence).squeeze().tolist()
    predicted_class = np.argmax(probabilities)

    class_names = {0: "📉 STRONG DOWN", 1: "🔹 FLAT", 2: "📈 STRONG UP"}

    print(f"Prediction: {class_names[predicted_class]}")
    print(f"DOWN Probability: {probabilities[0]:.2%}, FLAT Probability: {probabilities[1]:.2%}, UP Probability: {probabilities[2]:.2%}")


Epoch 1/2000, Loss: 1.0957, Test Accuracy: 35.45%
✅ Model saved with accuracy: 35.45%
Epoch 2/2000, Loss: 1.0912, Test Accuracy: 35.45%
Epoch 3/2000, Loss: 1.0902, Test Accuracy: 35.45%
Epoch 4/2000, Loss: 1.0899, Test Accuracy: 35.45%
Epoch 5/2000, Loss: 1.0894, Test Accuracy: 35.45%
Epoch 6/2000, Loss: 1.0890, Test Accuracy: 35.25%
Epoch 7/2000, Loss: 1.0884, Test Accuracy: 35.05%
Epoch 8/2000, Loss: 1.0886, Test Accuracy: 34.85%
Epoch 9/2000, Loss: 1.0877, Test Accuracy: 35.45%
Epoch 10/2000, Loss: 1.0876, Test Accuracy: 35.05%
Epoch 11/2000, Loss: 1.0868, Test Accuracy: 34.85%
Epoch 12/2000, Loss: 1.0852, Test Accuracy: 35.25%
Epoch 13/2000, Loss: 1.0844, Test Accuracy: 34.85%
Epoch 14/2000, Loss: 1.0828, Test Accuracy: 36.44%
✅ Model saved with accuracy: 36.44%
Epoch 15/2000, Loss: 1.0815, Test Accuracy: 34.65%
Epoch 16/2000, Loss: 1.0781, Test Accuracy: 35.25%
Epoch 17/2000, Loss: 1.0751, Test Accuracy: 35.05%
Epoch 18/2000, Loss: 1.0737, Test Accuracy: 34.65%
Epoch 19/2000, Loss

In [12]:
# Загружаем лучшую модель
model.load_state_dict(torch.load("best_model_3.pth"))
model.eval()

# Берем последние 20 свечей из df_fut
last_sequence = torch.tensor(
    df_fut['CANDLE_INT'].iloc[-20:].values, dtype=torch.long
    ).unsqueeze(0).to(device)

# Предсказание
with torch.no_grad():
    probabilities = model(last_sequence).squeeze().tolist()  # Вероятности для 3 классов
    predicted_class = np.argmax(probabilities)  # Класс с наибольшей вероятностью

    class_names = {0: "📉 STRONG DOWN", 1: "🔹 FLAT", 2: "📈 STRONG UP"}

    print(f"Prediction: {class_names[predicted_class]}")
    print(
        f"UP Probability: {probabilities[2]:.2%}, \n"
        f"FLAT Probability: {probabilities[1]:.2%}, \n"
        f"DOWN Probability: {probabilities[0]:.2%}"
        )

Prediction: 📉 STRONG DOWN
UP Probability: 36.59%, 
FLAT Probability: 13.56%, 
DOWN Probability: 49.86%
